In [1]:
import torch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class NARRE(nn.Module):
    def __init__(self, vocab_size, embedding_dim, review_maxlen, user_num, item_num, latent_dim, hidden_dims):
        super(NARRE, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.review_encoder = nn.LSTM(embedding_dim, latent_dim, num_layers=1, batch_first=True)
        self.user_embedding = nn.Embedding(user_num, latent_dim)
        self.item_embedding = nn.Embedding(item_num, latent_dim)
        self.attention = nn.Sequential(
            nn.Linear(latent_dim, hidden_dims),
            nn.Tanh(),
            nn.Linear(hidden_dims, 1),
            nn.Softmax(dim=1)
        )
        self.rating_predictor = nn.Sequential(
            nn.Linear(latent_dim, hidden_dims),
            nn.Tanh(),
            nn.Linear(hidden_dims, 1)
        )

    def forward(self, reviews, users, items):
        embedded_reviews = self.embedding(reviews)
        review_outputs, _ = self.review_encoder(embedded_reviews)
        user_embedded = self.user_embedding(users)
        item_embedded = self.item_embedding(items)
        attention_scores = self.attention(review_outputs).squeeze()
        attention_outputs = torch.bmm(attention_scores.unsqueeze(1), review_outputs).squeeze()
        latent_features = torch.cat((attention_outputs, user_embedded, item_embedded), dim=1)
        predicted_ratings = self.rating_predictor(latent_features)
        return predicted_ratings

# Exemplo de uso
vocab_size = 10000  # Tamanho do vocabulário (número de palavras únicas)
embedding_dim = 100  # Dimensão dos vetores de embedding
review_maxlen = 200  # Tamanho máximo das avaliações
user_num = 1000  # Número de usuários
item_num = 2000  # Número de itens
latent_dim = 50  # Dimensão dos vetores latentes
hidden_dims = 100  # Dimensão da camada oculta

# Criando uma instância do modelo
model = NARRE(vocab_size, embedding_dim, review_maxlen, user_num, item_num, latent_dim, hidden_dims)

# Definindo a função de perda e otimizador
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Exemplo de dados de entrada
reviews = torch.from_numpy(np.random.randint(0, vocab_size, (32, review_maxlen))).long()
users = torch.from_numpy(np.random.randint(0, user_num, (32,))).long()
items = torch.from_numpy(np.random.randint(0, item_num, (32,))).long()
ratings = torch.from_numpy(np.random.randint(1, 6, (32, 1))).float()

# Loop de treinamento
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    predicted_ratings = model(reviews, users, items)
    loss = loss_fn(predicted_ratings, ratings)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NARRE(nn.Module):
    def __init__(self, num_users, num_items, embed_dim, num_reviews):
        super(NARRE, self).__init__()

        self.user_embed = nn.Embedding(num_users, embed_dim)
        self.item_embed = nn.Embedding(num_items, embed_dim)
        self.review_embed = nn.EmbeddingBag(num_reviews, embed_dim)

        self.attention = nn.Linear(embed_dim, 1)
        self.prediction = nn.Linear(embed_dim * 2, 1)

    def forward(self, user_ids, item_ids, review_offsets):
        user_embeds = self.user_embed(user_ids)
        item_embeds = self.item_embed(item_ids)
        review_embeds = self.review_embed(review_offsets)

        # Calculate review attentions
        review_attentions = F.softmax(self.attention(review_embeds), dim=0)
        review_sum = torch.sum(review_embeds * review_attentions, dim=0)

        # Concatenate user and item embeddings
        user_item_embeds = torch.cat([user_embeds, item_embeds], dim=1)
        
        # Final prediction
        preds = self.prediction(user_item_embeds * review_sum)

        return preds.squeeze()
